In [1]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity
from bettmensch_ai_examples.gpt_1.src.model import GPT1Core, DecoderLayer, GPT1Pretrain

W0221 23:54:46.703000 9440 torch\distributed\elastic\multiprocessing\redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\scher\Repositories\bettmensch.ai-examples\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D50457590>
Base Component pipeline context: <bettmensch_ai.pi

In [2]:
def time_model(n, batch_size=5,layer_only: bool = True, use_gpu=False,sort_by="self_cpu_time_total"):

    mask = torch.ones((batch_size,512),dtype=torch.bool)

    if layer_only:
        inputs = torch.rand(batch_size, 512, 768,dtype=torch.half)
        model = DecoderLayer()
        model.set_nest_level()
    else:
        inputs = torch.randint(low=0,high=30000,size=(batch_size, 512))
        model = GPT1Core(30000)
        
    if use_gpu:
        gpu_device = torch.device("cuda")
        inputs = inputs.to(gpu_device)
        mask = mask.to(gpu_device)
        model.to(gpu_device)

    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        with record_function("model_inference"):
            for k in range(n):
                model(inputs, mask)

    print(prof.key_averages().table(sort_by=sort_by, row_limit=550))

    return prof

## Layer level comparison

In [3]:
prof = time_model(n=10,batch_size=2,layer_only=True, use_gpu=False,sort_by="self_cpu_time_total")

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::addmm        88.82%       69.209s        88.83%       69.215s        1.730s       69.218s        88.83%       69.224s        1.731s            40  
                  aten::bmm        10.37%        8.082s        10.40%        8.104s     405.196ms        8.088s        10.38%        8.107s     405.350ms            20  
             aten::_softmax         0.15%     116.521ms         0.15%     116.521ms      11.652ms     116.900ms         0.15%     116.900ms      11.6

In [7]:
prof = time_model(n=100,batch_size=16,layer_only=True, use_gpu=True,sort_by="self_cuda_time_total")

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::addmm         1.12%      28.835ms         1.38%      35.431ms      88.578us        1.068s        41.16%        1.076s       2.689ms           400  
       aten::native_dropout         0.73%      18.871ms         1.31%      33.697ms      67.394us     280.123ms        10.80%     318.371ms     636.742us           500  
                aten::copy_        85.10%        2.190s        85.10%        2.190s       2.747ms     250.281ms         9.65%     250.281ms     314.0

## Model level comparison

In [3]:
time_model(n=100,batch_size=16,layer_only=False,use_gpu=True,sort_by="self_cuda_time_total")

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::addmm         1.29%     581.193ms         1.99%     898.208ms     187.127us       17.981s        39.70%       18.064s       3.763ms          4800  
       aten::native_dropout         0.81%     365.363ms         1.26%     569.903ms      93.427us        6.785s        14.98%        7.218s       1.183ms          6100  
                aten::copy_        81.80%       36.947s        81.80%       36.947s       3.862ms        4.710s        10.40%        4.710s     492.2

In [3]:
time_model(n=100,batch_size=16,layer_only=False,karpathy=True,use_gpu=True,sort_by="self_cuda_time_total")

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::addmm         2.00%     863.521ms         2.46%        1.061s     220.938us       17.226s        39.94%       17.313s       3.607ms          4800  
       aten::native_dropout         1.56%     674.079ms         2.81%        1.210s     198.343us        6.467s        14.99%        6.924s       1.135ms          6100  
                aten::copy_        71.44%       30.815s        71.44%       30.815s       3.217ms        4.537s        10.52%        4.537s     473.7

In [4]:
gpu_device = torch.device("cuda")

model = DecoderLayer()
model.set_nest_level()
model.to(gpu_device)

inputs = torch.rand(1, 512, 768,dtype=torch.half).to(gpu_device)
mask = torch.ones((1,512),dtype=torch.bool).to(gpu_device)

outputs = model(inputs,mask)
outputs.size(), outputs

(torch.Size([1, 512, 768]),
 tensor([[[-0.5049, -2.5391,  0.5952,  ..., -0.0470, -0.4937,  1.8857],
          [ 0.3708, -0.8301,  2.7305,  ..., -0.0000,  0.7598,  0.1093],
          [ 0.0000, -0.1092, -0.3325,  ...,  1.6611, -1.0459, -0.2112],
          ...,
          [ 2.0527,  0.0000, -0.3018,  ...,  0.0000,  0.0706,  0.1324],
          [ 1.6699, -1.2598,  0.1018,  ..., -1.2334,  1.2178, -1.6279],
          [ 1.1924,  0.6562,  0.0355,  ...,  0.0000, -0.0049,  0.3257]]],
        device='cuda:0', dtype=torch.float16, grad_fn=<NativeDropoutBackward0>))

In [5]:
gpu_device = torch.device("cuda")

model = GPT1Core(30000)
model.to(gpu_device)

inputs = torch.randint(low=0, high=30000, size=(1, 512)).to(gpu_device)
mask = torch.ones((1,512),dtype=torch.bool).to(gpu_device)

outputs = model(inputs,mask)
outputs.size(), outputs

(torch.Size([1, 512, 768]),
 tensor([[[-0.7222, -0.0000,  0.4094,  ...,  2.3184,  0.9126, -0.3777],
          [-2.1016,  0.3618,  0.2703,  ...,  0.1775,  0.0360, -0.0820],
          [ 0.5449,  0.2352, -0.0000,  ...,  0.0692, -0.9214, -0.6865],
          ...,
          [ 0.6113,  0.2185, -0.2742,  ..., -0.1758, -0.0150, -0.0599],
          [-0.1068,  0.9580,  1.0029,  ...,  0.0000, -0.1261,  0.1019],
          [ 3.0938, -0.4995, -0.4680,  ..., -0.6748,  0.0461,  0.3140]]],
        device='cuda:0', dtype=torch.float16, grad_fn=<NativeDropoutBackward0>))

In [2]:
gpu_device = torch.device("cuda")

model = GPT1Pretrain(30000)
model.to(gpu_device)

inputs = torch.randint(low=0, high=30000, size=(1, 512)).to(gpu_device)
mask = torch.ones((1,512),dtype=torch.bool).to(gpu_device)

outputs, loss = model(inputs,mask)
outputs.size(), outputs

(torch.Size([1, 512, 30000]),
 tensor([[[-0.0814,  0.4971, -0.1242,  ..., -0.2145,  0.0060,  0.5791],
          [-0.2820,  0.0219, -0.1628,  ...,  0.8433, -0.1349,  0.4685],
          [-0.3379, -1.1406,  0.3604,  ...,  1.0996,  0.2864,  0.0953],
          ...,
          [ 1.2432, -0.1678,  0.4783,  ...,  1.0586,  0.6890, -0.1908],
          [-0.6265,  0.5337, -0.2869,  ..., -0.2651, -0.6992,  0.1315],
          [ 0.1234, -0.1227, -0.5156,  ...,  0.1487, -0.5483,  0.1032]]],
        device='cuda:0', dtype=torch.float16, grad_fn=<ViewBackward0>))